# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [2]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

In [4]:
K_LFM_ITEMS = 2
K_POPULAR_ITEMS = 1
K_ITEMS = 10

### Retrieve data

In [5]:
import pandas as pd
# path config

df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=10000)
df_valid.dropna()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
9995,eb82748abb8faabca586f788a10887170105475b16e575...,Logged,['2277cab9-798b-4b0e-8f7c-cb2862ad3e4c'],[1660675022675]
9996,432bd2d2f562efcc59a3943b943b8783788c3526801761...,Logged,['35e20551-659a-415f-95fe-7acbb89b69f6'],[1660571720373]
9997,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,Logged,['5f05a0e2-5e92-47e3-b8c2-8d15ee772db3'],[1660665278986]
9998,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,Logged,['f52361e4-9206-49c4-8117-2451c0b0c6f1'],[1660677752418]


In [6]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.122610,very-old


In [7]:
import pickle

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))
loaded_interactions_shape = pickle.load(open('artifacts/interactions_shape.pkl', 'rb'))

loaded_n_users, loaded_n_items = loaded_interactions_shape

### Make predictions to known and unknowm on same recommendation function with pkls

In [8]:
from utils.custom_treat_data_funcs import explode_df_columns

cols_to_explode = ["history", "timestampHistory"]
cols_and_id = cols_to_explode.copy()
cols_and_id.insert(0,"userId")
cols_and_id.append("userType")
cols_and_id = tuple(cols_and_id)
exploded_df = explode_df_columns(df_valid.loc[:,cols_and_id], cols_to_explode)

In [9]:
df_valid

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
9995,eb82748abb8faabca586f788a10887170105475b16e575...,Logged,['2277cab9-798b-4b0e-8f7c-cb2862ad3e4c'],[1660675022675]
9996,432bd2d2f562efcc59a3943b943b8783788c3526801761...,Logged,['35e20551-659a-415f-95fe-7acbb89b69f6'],[1660571720373]
9997,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,Logged,['5f05a0e2-5e92-47e3-b8c2-8d15ee772db3'],[1660665278986]
9998,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,Logged,['f52361e4-9206-49c4-8117-2451c0b0c6f1'],[1660677752418]


In [10]:
exploded_df

,userId,history,timestampHistory,userType
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1660533136590,Logged
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,01c59ff6-fb82-4258-918f-2910cb2d4c52,1660672113513,Logged
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,77901133-aee7-4f7b-afc0-652231d76fe9,1660556860253,Logged
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,1660561649242,Logged
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,1660533830245,Logged
...,...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c,1660675022675,Logged
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,35e20551-659a-415f-95fe-7acbb89b69f6,1660571720373,Logged
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3,1660665278986,Logged
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,f52361e4-9206-49c4-8117-2451c0b0c6f1,1660677752418,Logged


In [11]:
exploded_df["check_hifens"] = exploded_df["history"].str.split('-').apply(lambda x : len(x) == 5).astype("bool")
exploded_df["check_size"] = exploded_df["history"].str.replace(r'-', '', regex=True).apply(lambda x : len(x) == 32).astype("bool")
exploded_df["check_chars"] = exploded_df["history"].str.replace(r'-', '', regex=True).str.replace(r'[a-f0-9]', '', regex=True).apply(lambda x : x == '').astype("bool")
exploded_df["check_history"] = (exploded_df["check_hifens"] & exploded_df["check_size"] & exploded_df["check_chars"]).astype("bool")
exploded_df["check_history"].value_counts()

check_history
True     18468
False      573
Name: count, dtype: int64

In [12]:
exploded_df_valid = exploded_df[~exploded_df["check_history"]==False][["userId","userType","history"]]
exploded_df_valid

,userId,userType,history
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85
...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,Logged,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,Logged,35e20551-659a-415f-95fe-7acbb89b69f6
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,Logged,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,Logged,f52361e4-9206-49c4-8117-2451c0b0c6f1


In [13]:
count_histories = exploded_df_valid.groupby(["history"]).size().sort_values(ascending=False).astype(dtype="UInt16")
count_histories

history
9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6    813
4c3d47a1-6f4b-424f-8944-6c227e686c5c    614
eb23272d-8e6c-479d-b972-eabeb5f6f3dd    536
d730c4a6-e8f6-4fde-b73a-afbe148479cd    274
aeab0e46-f1e4-41e9-821b-571255c41f69    246
                                       ... 
70d987ad-a50a-4490-815a-efbeb0ecab2c      1
70e337bb-2d53-4f51-977f-4ee636ea3eb4      1
6fa217fb-92a1-48d1-bf8e-a75369f34aa9      1
6faaf970-bbbf-43cc-b3ec-3ed5d53d6098      1
708f0b7e-3158-403c-b459-a67ab2188eca      1
Length: 6998, dtype: UInt16

In [14]:
top_k_popular_histories = count_histories.head(K_POPULAR_ITEMS)
top_k_popular_histories = list(set(top_k_popular_histories.keys()))

In [15]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [16]:
def sample_recommendation_by_title(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]
    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories))

    print("Top 5 recommended items:")

    for x in top_k_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["title"].values[0])


In [17]:
def get_recommended_history_list(user_hash,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top 5 recommendations from the list are returned.
    """
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_k_indices = np.argsort(-scores)[:K_LFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    top_k_items_lfm = [item_id_map_reverse[i] for i in top_k_indices]
    top_k_items = list(set(top_k_items_lfm + top_k_popular_histories))

    return top_k_items

In [18]:
# # Testing for just one user
# user = 38
# user_feature_list = [df_valid["userType"].iloc[user]]
# user_hash = df_valid["userId"].iloc[user]
# validation_history_hashes = df_valid["history"].iloc[user]

# recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
# print(recommeded_histories)
# print(validation_history_hashes)
# print(validation_history_hashes in recommeded_histories)

In [19]:
def count_valid_recommendations(validation_history_hashes, recommeded_histories):
    """
    This function receives 
    * The `validation_history_hashes` (which is a list of histories contained in the "validacao.csv")
    * The `recommeded_histories` (which are the recommended histories/items by the model)
    Then, it verifies how many recommended items match with the validation histories.
    """
    count_valids = 0
    for valid_hist in validation_history_hashes:
        if valid_hist in recommeded_histories:
            count_valids = count_valids+1
    return count_valids

In [20]:
# for index, row in df_valid.iterrows():
#     """
#     For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
#     Stores the results on the columns `recommended_hists` `matched_recommendations`.
#     """
#     user_feature_list = [row["userType"]]
#     user_hash = row["userId"]
#     validation_history_hashes = row["history"]

#     recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
#     num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

#     row["recommended_hists"] = recommended_hist
#     row["matched_recommendations"] = num_valid_recommendations

In [21]:
exploded_df_valid = exploded_df_valid.head(1000)

exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.apply(lambda x :
    get_recommended_history_list(
        x.userId,
        [x.userType],
        loaded_item_id_map_reverse,
        loaded_user_feature_map,
        loaded_user_id_map,
        loaded_model),
        axis=1)

/tmp/ipykernel_306532/1618428163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df_valid.loc[:,"recommended_hists"] = exploded_df_valid.apply(lambda x :


In [22]:
exploded_df_valid["matched_recommendations"] = exploded_df_valid.apply(lambda x :
    count_valid_recommendations(x.history, x.recommended_hists),
    axis=1
    )

/tmp/ipykernel_306532/2697419088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df_valid["matched_recommendations"] = exploded_df_valid.apply(lambda x :


In [23]:
exploded_df_valid.describe()

,matched_recommendations
count,1000.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [24]:
exploded_df_valid

,userId,userType,history,recommended_hists,matched_recommendations
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
...,...,...,...,...,...
1029,a98d150b8cf967b0fcb8d117a4e5531ab6f051595053c4...,Logged,be81f38c-fc94-4d30-90cc-4136b01dc83d,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1030,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,9d3770aa-e8bb-44a6-b2bb-ac511bf1033a,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1031,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1032,73d4fd880933325b82bed1890b7f9322a9222e40e5440c...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0


In [25]:
user = 1
user_feature_list = [exploded_df_valid["userType"].iloc[user]]
user_hash = exploded_df_valid["userId"].iloc[user]
validation_history_hashes = exploded_df_valid["history"].iloc[user]

print(user_hash)
print(user_feature_list)

recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
print(recommeded_histories)
print(validation_history_hashes)
print(validation_history_hashes in recommeded_histories)

e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9
['Logged']
['30509cec-b02e-488c-9e43-edeaab751062', 'c1c17f5e-f97c-4911-ba24-358642b611db', '9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6']
01c59ff6-fb82-4258-918f-2910cb2d4c52
False


In [26]:
exploded_df_valid[exploded_df_valid["matched_recommendations"]>0]

,userId,userType,history,recommended_hists,matched_recommendations


In [28]:
exploded_df_valid

,userId,userType,history,recommended_hists,matched_recommendations
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
...,...,...,...,...,...
1029,a98d150b8cf967b0fcb8d117a4e5531ab6f051595053c4...,Logged,be81f38c-fc94-4d30-90cc-4136b01dc83d,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1030,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,9d3770aa-e8bb-44a6-b2bb-ac511bf1033a,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1031,0c564af27eaac70be62f377fc681e2e3236b58a9d57ff8...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0
1032,73d4fd880933325b82bed1890b7f9322a9222e40e5440c...,Logged,72014099-6ee5-4c3a-9f96-7ed0b7461edb,"[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5...",0


In [29]:
exploded_df_valid["recommended_hists"].value_counts()

recommended_hists
[30509cec-b02e-488c-9e43-edeaab751062, c1c17f5e-f97c-4911-ba24-358642b611db, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]    922
[09fca7c0-d27e-48b6-9210-9e19abab2f45, c22f9540-546c-499c-8c8a-4fa281ed7377, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]     44
[cd32d988-cfca-421a-ab0d-cfcee0d806d3, c22f9540-546c-499c-8c8a-4fa281ed7377, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]     10
[c22f9540-546c-499c-8c8a-4fa281ed7377, fb9b10c0-6d5b-4353-89e2-e805f68e2e3a, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]      4
[cd32d988-cfca-421a-ab0d-cfcee0d806d3, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, 01c814ce-12aa-42a3-9eec-a5e2cf9a522b]      3
[c22f9540-546c-499c-8c8a-4fa281ed7377, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, 89027ad6-7424-4c24-aeec-e932d153ca5f]      3
[c6ced325-bbbb-45c4-8a39-df535be3655a, b117add4-b07b-4a07-88a2-5e91ae386927, 9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6]      3
[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, c42979bd-39fd-4a0b-a0f6-192bf764edbc, db5fc53f-25f9-4346-bf36-55cb623cfdaf]      2
[266b2